## Part 1: Preprocessing

In [33]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [34]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [35]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()



,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [62]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 
                    'HourlyRate', 'JobInvolvement', 
                    'JobLevel', 'JobSatisfaction', 'WorkLifeBalance', 'YearsInCurrentRole'] 

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes



Age                        int64
DistanceFromHome           int64
Education                  int64
EnvironmentSatisfaction    int64
HourlyRate                 int64
JobInvolvement             int64
JobLevel                   int64
JobSatisfaction            int64
WorkLifeBalance            int64
YearsInCurrentRole         int64
dtype: object

In [63]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [64]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_numeric = pd.get_dummies(X_df)
X_numeric.head()


,Age,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,WorkLifeBalance,YearsInCurrentRole
0,41,1,2,2,94,3,2,4,1,4
1,49,8,1,3,61,2,2,2,3,7
2,37,2,2,4,92,2,1,3,3,0
3,33,3,4,4,56,3,1,3,3,7
4,27,2,1,1,40,3,1,2,3,2


In [65]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [67]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded = encoder.transform(y_train[['Department']])
y_test_encoded = encoder.transform(y_test[['Department']])


In [68]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [69]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [70]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(y_train_encoded.shape[1], activation='softmax', name='department_output')(department_hidden_layer)



In [71]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition_encoded.shape[1], activation='softmax', name='attrition_output')(attrition_hidden_layer)


In [72]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'categorical_crossentropy'},
              metrics=['accuracy', 'accuracy'])

# Summarize the model
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │        704 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 32)        │      2,080 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 16)        │        528 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 16)        │        528 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         51 │ dense_6[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         34 │ dense_7[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,925 (15.33 KB)

 Trainable params: 3,925 (15.33 KB)

 Non-trainable params: 0 (0.00 B)

In [73]:
# Train the model
history = model.fit(X_train_scaled, {'department_output': y_train_encoded, 'attrition_output': y_train_attrition_encoded}, epochs=100, 
                    batch_size=32, 
                    validation_data=(X_test_scaled, {'department_output': y_test_encoded, 'attrition_output': y_test_attrition_encoded}))



Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - attrition_output_accuracy: 0.7820 - attrition_output_loss: 0.5680 - department_output_accuracy: 0.2631 - department_output_loss: 1.2078 - loss: 1.7759 - val_attrition_output_accuracy: 0.8673 - val_attrition_output_loss: 0.3815 - val_department_output_accuracy: 0.6224 - val_department_output_loss: 0.9023 - val_loss: 1.3127
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - attrition_output_accuracy: 0.8368 - attrition_output_loss: 0.4430 - department_output_accuracy: 0.6462 - department_output_loss: 0.8560 - loss: 1.2990 - val_attrition_output_accuracy: 0.8673 - val_attrition_output_loss: 0.3604 - val_department_output_accuracy: 0.6667 - val_department_output_loss: 0.7787 - val_loss: 1.1702
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8202 - attrition_output_loss: 0.4458 - department_output_accuracy: 0.6304 - department_output_loss: 0.7998 - loss: 1.2457 - val_attrition_output_accuracy: 0.8673 

In [75]:
# Evaluate the model with the testing data
evaluation = model.evaluate(X_test_scaled, {'department_output': y_test_encoded, 'attrition_output': y_test_attrition_encoded}, verbose=1)

print(evaluation)

test_loss = evaluation[0]
department_output_loss = evaluation[1]
attrition_output_loss = evaluation[2]

print(f"Test Loss: {test_loss}")
print(f"Test Department Output Loss: {department_output_loss}")
print(f"Test Attrition Output Loss: {attrition_output_loss}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - attrition_output_accuracy: 0.7734 - attrition_output_loss: 1.0351 - department_output_accuracy: 0.5508 - department_output_loss: 1.7487 - loss: 2.8020
[2.4168577194213867, 1.4881377220153809, 0.828650176525116, 0.7891156673431396, 0.5986394286155701]
Test Loss: 2.4168577194213867
Test Department Output Loss: 1.4881377220153809
Test Attrition Output Loss: 0.828650176525116


In [77]:
# Print the accuracy for both department and attrition
department_pred = model.predict(X_test_scaled)[0]
department_accuracy = np.mean(np.argmax(department_pred, axis=1) == np.argmax(y_test_encoded, axis=1))

attrition_pred = model.predict(X_test_scaled)[1]
attrition_accuracy = np.mean(np.argmax(attrition_pred, axis=1) == np.argmax(y_test_attrition_encoded, axis=1))

print(f"Test Department Output Accuracy: {department_accuracy}")
print(f"Test Attrition Output Accuracy: {attrition_accuracy}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Test Department Output Accuracy: 0.5986394557823129
Test Attrition Output Accuracy: 0.7891156462585034


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. **Is accuracy the best metric to use on this data? Why or why not?**

Accuracy is **not** the best metric to use on this data, as it not account for class imbalances. For example, if attrition (the number of employees who leave) at the company is significantly lower than those who stay, then accuracy could be misleading. Precision, recall, and F1-score can provide more insight into the model's performance, especially for minority classes. They offer a more comprehensive view of the model's effectiveness by considering the true positive, false positive, and false negative rates.  Therefore, accuracy is not the best metric.
2. **What activation functions did you choose for your output layers, and why?**
We chose softmax for both department_output and attrition_output. Softmax is used for multi-class classification in the department_output to ensure the outputs total to 1, and represent probabilities for each class. For attrition_output, although it’s a binary classification, softmax is used to get probabilities of each class (Yes/No). This allows us to interpret the outputs as the likelihood of each class, facilitating better decision-making based on predicted probabilities.
3. **Can you name a few ways that this model might be improved?**

* **More Data:** Increase the amount of training data to improve model generalization. collaborating with other sources for additional data can help the model learn better.

* **Feature Engineering:** Add or modify features to provide better input data. For example, create new features like the ratio of total working years to years at the company, or encode tenure in months rather than years.

* **Hyperparameter Tuning:** Adjust learning rates, batch sizes, and the number of neurons/layers. Try different values for these parameters to find the best combination that improves model performance.

* **Regularization:** Implement dropout or L2 regularization to prevent overfitting. Dropout randomly ignores some neurons during training to make the model more robust, while L2 regularization adds a penalty for large weights.

* **Different Architectures:** Experiment with different neural network architectures or other machine learning models. For example, try using convolutional neural networks (CNNs) for feature extraction or ensemble methods like random forests and gradient boosting for improved accuracy.